<a href="https://colab.research.google.com/github/johyunkang/py_pandas/blob/main/python_ml_perfect_guide_07_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 07 군집화


### 01 K-평균 알고리즘 이해
K-평균 군집 중심점(Centroid)이라는 특정한 임의의 지점을 선택해 해당 중심에 가장 가까운 포인트들을 선택하는 군집화 기법

![k-mean-clustering](https://user-images.githubusercontent.com/291782/142011995-c697fbde-af31-4f40-8c27-994fc3addb18.png)

K-평균의 장점
- 일반적인 군집화에서 가장 많이 활용되는 알고리즘
- 알고리즘이 쉽고 간결  

&nbsp;  
K-평균의 단점
- 거리기반 알고리즘으로 속성의 개수가 매무 많을 경우 군집화 정확도가 떨어짐. (이를 위해 PCA로 차원감소 필요 할 수도 있음)
- 반복을 수행하는데, 반복 횟수가 많을 경우 수행 시간이 매우 느려짐
- 몇 개의 군집(cluster)을 선택해야 할지 가이드하기가 어려움

In [ ]:
# p.410 , 2